# Data Loading

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
folder = 'Stimulus_Screen_Data/variables_for_fitting_activity/'
HZ = 10


def analyze_neural_activity_around_tap(tap_events, neural_data, tap_index=0, 
                                     time_before=10, time_after=10, hz=1000,
                                     neuron_selection='auto', neuron_indices=None):
    """
    Analyze neural activity around a specific tap event.
    
    Parameters:
    -----------
    tap_events : array-like
        Array of tap event timestamps (in samples)
    neural_data : numpy.ndarray
        Neural data matrix (neurons x time)
    tap_index : int, default=0
        Index of the tap event to analyze
    time_before : float, default=10
        Time window before tap (in seconds)
    time_after : float, default=10
        Time window after tap (in seconds)
    hz : int, default=1000
        Sampling rate (Hz)
    neuron_selection : str, default='auto'
        'auto' to pick neurons with highest variance, 'manual' to use neuron_indices
    neuron_indices : tuple, optional
        Tuple of (neuron1_idx, neuron2_idx) for manual selection
    
    Returns:
    --------
    dict
        Dictionary containing analysis results and plot data
    """
    
    # Input validation
    if tap_index >= len(tap_events):
        raise ValueError(f"tap_index {tap_index} is out of range for tap_events length {len(tap_events)}")
    
    # Get the tap time
    tap_time = tap_events[tap_index]
    
    # Convert time windows to samples
    time_before_samples = int(time_before * hz)
    time_after_samples = int(time_after * hz)
    
    # Check if we have enough data around this tap
    if tap_time < time_before_samples or tap_time + time_after_samples >= neural_data.shape[1]:
        raise ValueError(f"Not enough data around tap at sample {tap_time}")
    
    # Extract the time window around this tap
    start_idx = tap_time - time_before_samples
    end_idx = tap_time + time_after_samples
    neural_window = neural_data[:, start_idx:end_idx]
    
    # Select neurons to analyze
    if neuron_selection == 'auto':
        neuron_responses = np.var(neural_window, axis=1)
        neuron1_idx, neuron2_idx = np.argsort(neuron_responses)[::-1][:2]
    elif neuron_selection == 'manual':
        if neuron_indices is None:
            raise ValueError("neuron_indices must be provided when neuron_selection='manual'")
        neuron1_idx, neuron2_idx = neuron_indices
    else:
        raise ValueError("neuron_selection must be 'auto' or 'manual'")
    
    # Extract the two neurons' activity
    neuron1_activity = neural_window[neuron1_idx, :]
    neuron2_activity = neural_window[neuron2_idx, :]
    
    # Create time points in absolute time (seconds)
    absolute_time_points = (np.arange(len(neuron1_activity)) + start_idx) / hz
    
    # Calculate tap positions and times within the window
    tap_list = []
    tap_times_absolute = []
    tap_indices_absolute = []
    
    for i, tap in enumerate(tap_events):
        relative_position = (tap - tap_time) + time_before_samples
        if 0 <= relative_position < neural_window.shape[1]:
            tap_list.append(relative_position)
            tap_times_absolute.append(tap / hz)
            tap_indices_absolute.append(i)
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Top left: Neuron 1 vs time
    axes[0,0].plot(absolute_time_points, neuron1_activity, 'b-', linewidth=2)
    for local_idx, (tap, tap_time_abs, abs_idx) in enumerate(zip(tap_list, tap_times_absolute, tap_indices_absolute)):
        tap_time_on_plot = absolute_time_points[tap]
        axes[0,0].axvline(tap_time_on_plot, color='r', linestyle='--', linewidth=2, 
                         label=f'Tap #{abs_idx} (t={tap_time_abs:.2f}s)')
    
    axes[0,0].set_xlabel('Time (seconds)')
    axes[0,0].set_ylabel('Firing Rate')
    axes[0,0].set_title(f'Neuron {neuron1_idx} vs Time')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].legend()
    
    # Top right: Neuron 2 vs time
    axes[0,1].plot(absolute_time_points, neuron2_activity, 'g-', linewidth=2)
    for local_idx, (tap, tap_time_abs, abs_idx) in enumerate(zip(tap_list, tap_times_absolute, tap_indices_absolute)):
        tap_time_on_plot = absolute_time_points[tap]
        axes[0,1].axvline(tap_time_on_plot, color='r', linestyle='--', linewidth=2, 
                         label=f'Tap #{abs_idx} (t={tap_time_abs:.2f}s)')
    
    axes[0,1].set_xlabel('Time (seconds)')
    axes[0,1].set_ylabel('Firing Rate')
    axes[0,1].set_title(f'Neuron {neuron2_idx} vs Time')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].legend()
    
    # Bottom: Trajectory plot (spanning both bottom subplots)
    axes[1,0].remove()
    axes[1,1].remove()
    ax_traj = fig.add_subplot(2, 1, 2)
    
    # Plot the trajectory with time-based coloring
    scatter = ax_traj.scatter(neuron1_activity, neuron2_activity, c=absolute_time_points, 
                             cmap='viridis', s=30, alpha=0.7)
    
    # Mark the tap times
    for local_idx, (tap, tap_time_abs, abs_idx) in enumerate(zip(tap_list, tap_times_absolute, tap_indices_absolute)):
        ax_traj.plot(neuron1_activity[tap], neuron2_activity[tap], 'ro', 
                    markersize=12, label=f'Tap #{abs_idx} (t={tap_time_abs:.2f}s)', 
                    markeredgecolor='white', markeredgewidth=2)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax_traj)
    cbar.set_label('Time (seconds)')
    
    ax_traj.set_xlabel(f'Neuron {neuron1_idx} Firing Rate')
    ax_traj.set_ylabel(f'Neuron {neuron2_idx} Firing Rate')
    ax_traj.set_title('Neural State Trajectory (Colored by Time)')
    ax_traj.legend()
    ax_traj.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary information
    print(f"Analyzing tap #{tap_index} at sample {tap_time}")
    print(f"Selected neurons: {neuron1_idx} and {neuron2_idx}")
    print(f"Taps in window at samples: {tap_list}")
    print(f"Tap times in seconds: {[f'{t:.2f}s' for t in tap_times_absolute]}")
    print(f"Absolute tap indices: {tap_indices_absolute}")
    
    # Return results dictionary
    return {
        'neuron1_idx': neuron1_idx,
        'neuron2_idx': neuron2_idx,
        'neuron1_activity': neuron1_activity,
        'neuron2_activity': neuron2_activity,
        'absolute_time_points': absolute_time_points,
        'tap_list': tap_list,
        'tap_times_absolute': tap_times_absolute,
        'tap_indices_absolute': tap_indices_absolute,
        'neural_window': neural_window,
        'start_idx': start_idx,
        'end_idx': end_idx
    }



def clean_neural_data(all_neural_data, good_exps, zero_threshold=0.5, verbose=True, return_structure='nested'):
    """
    Clean neural data by removing neurons with high zero proportions.
    
    Parameters:
    -----------
    all_neural_data : dict
        Nested dictionary: all_neural_data[animal][stimuli] = 2D array
        where rows are neurons and columns are time points/trials
    good_exps : dict
        Dictionary mapping animals to lists of stimuli
    zero_threshold : float, default=0.5
        Threshold for proportion of zeros (neurons above this are removed)
    verbose : bool, default=True
        Whether to print progress information
    return_structure : str, default='nested'
        'nested' returns cleaned_data[animal][stimuli] structure
        'dict' returns {'cleaned_data': ..., 'bad_neuron_indices': ...}
    
    Returns:
    --------
    dict
        If return_structure='nested': nested dict with cleaned_data[animal][stimuli]
        If return_structure='dict': dict with 'cleaned_data' and 'bad_neuron_indices' keys
    """
    
    def get_high_zero_neurons(data, threshold):
        """Helper function to find neurons with high zero proportions."""
        zero_proportions = np.mean(data == 0, axis=1)
        return np.where(zero_proportions > threshold)[0]
    
    # Find bad neurons for each animal (across all stimuli)
    all_bad_neurons = {}
    
    for animal in good_exps:
        animal_bad_inds = []
        
        for stimuli in good_exps[animal]:
            data = all_neural_data[animal][stimuli]
            bad_inds = get_high_zero_neurons(data, zero_threshold)
            animal_bad_inds.extend(bad_inds)
        
        # Remove duplicates
        animal_bad_inds = np.unique(animal_bad_inds)
        all_bad_neurons[animal] = animal_bad_inds
        
        if verbose:
            print(f"Animal {animal}: {len(animal_bad_inds)} neurons with >{zero_threshold*100}% zeros")
    
    # Clean the data by removing bad neurons
    all_exps_cleaned = {}
    
    for animal in good_exps:
        bad_indices = all_bad_neurons[animal]
        all_exps_cleaned[animal] = {}  # Create nested structure
        
        for stimuli in good_exps[animal]:
            data = all_neural_data[animal][stimuli]
            
            # Keep only good neurons
            good_indices = np.setdiff1d(np.arange(data.shape[0]), bad_indices)
            cleaned_data = data[good_indices, :]
            
            all_exps_cleaned[animal][stimuli] = cleaned_data
            
            if verbose:
                print(f"Animal {animal}, Stimuli {stimuli}: "
                      f"{data.shape[0]} -> {cleaned_data.shape[0]} neurons "
                      f"(removed {len(bad_indices)})")
    
    # Return based on requested structure
    if return_structure == 'nested':
        return all_exps_cleaned
    else:
        return {
            'cleaned_data': all_exps_cleaned,
            'bad_neuron_indices': all_bad_neurons
        }
    """
    Clean neural data by removing neurons with high zero proportions.
    
    Parameters:
    -----------
    all_neural_data : dict
        Nested dictionary: all_neural_data[animal][stimuli] = 2D array
        where rows are neurons and columns are time points/trials
    good_exps : dict
        Dictionary mapping animals to lists of stimuli
    zero_threshold : float, default=0.5
        Threshold for proportion of zeros (neurons above this are removed)
    verbose : bool, default=True
        Whether to print progress information
    
    Returns:
    --------
    dict
        Dictionary with keys:
        - 'cleaned_data': dict mapping (animal, stimuli) tuples to cleaned arrays
        - 'bad_neuron_indices': dict mapping animals to arrays of bad neuron indices
    """
    
    def get_high_zero_neurons(data, threshold):
        """Helper function to find neurons with high zero proportions."""
        zero_proportions = np.mean(data == 0, axis=1)
        return np.where(zero_proportions > threshold)[0]
    
    # Find bad neurons for each animal (across all stimuli)
    all_bad_neurons = {}
    
    for animal in good_exps:
        animal_bad_inds = []
        
        for stimuli in good_exps[animal]:
            data = all_neural_data[animal][stimuli]
            bad_inds = get_high_zero_neurons(data, zero_threshold)
            animal_bad_inds.extend(bad_inds)
        
        # Remove duplicates
        animal_bad_inds = np.unique(animal_bad_inds)
        all_bad_neurons[animal] = animal_bad_inds
        
        if verbose:
            print(f"Animal {animal}: {len(animal_bad_inds)} neurons with >{zero_threshold*100}% zeros")
    
    # Clean the data by removing bad neurons
    all_exps_cleaned = {}
    
    for animal in good_exps:
        bad_indices = all_bad_neurons[animal]
        
        for stimuli in good_exps[animal]:
            data = all_neural_data[animal][stimuli]
            
            # Keep only good neurons
            good_indices = np.setdiff1d(np.arange(data.shape[0]), bad_indices)
            cleaned_data = data[good_indices, :]
            
            all_exps_cleaned[(animal, stimuli)] = cleaned_data
            
            if verbose:
                print(f"Animal {animal}, Stimuli {stimuli}: "
                      f"{data.shape[0]} -> {cleaned_data.shape[0]} neurons "
                      f"(removed {len(bad_indices)})")
    
    return {
        'cleaned_data': all_exps_cleaned,
        'bad_neuron_indices': all_bad_neurons
    }


def load_pickle(fn):
    full_fp = f'{folder}{fn}'
    with open(full_fp, 'rb') as file:
        data = pickle.load(file)
    return data


def plot_heat_touch(neural_data, touches, title='Response to touch', vmax=None, ax=None):
    """
    Plot neural data heatmap with touch indicators.
    
    Parameters:
    -----------
    neural_data : array-like
        2D array of neural data
    touches : array-like
        Array of touch timepoints
    title : str
        Plot title
    vmax : float, optional
        Maximum value for colormap
    ax : matplotlib.axes.Axes, optional
        Axes to plot on. If None, creates new figure
        
    Returns:
    --------
    tuple : (ax, im)
        The axes object and image object for colorbar creation
    """
    # If no axes provided, create a new figure
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
        show_plot = True
    else:
        show_plot = False
    
    # Set up imshow parameters
    imshow_kwargs = {'aspect': 'auto', 'origin': 'lower', 
                     'extent': [0, neural_data.shape[1]/HZ, 0, neural_data.shape[0]]}
    if vmax is not None:
        imshow_kwargs['vmax'] = vmax
    
    # Create the heatmap
    im = ax.imshow(neural_data, **imshow_kwargs)
    
    # Set limits and labels
    ax.set_ylim(0, neural_data.shape[0])
    ax.set_ylabel('neuron index')
    ax.set_xlim((0, neural_data.shape[1]/HZ))
    ax.set_xlabel('time (s)')
    ax.set_title(title)
    
    # Add touch indicators
    if not touches:
        print('Warning: touch is empty')
    else: 
        for i, touch in enumerate(touches):
            if i == 0:
                ax.axvline(touch/HZ, label='touch', color='red', alpha=0.7)
            else:
                ax.axvline(touch/HZ, color='red', alpha=0.7)
        
        ax.legend()
    
    # Only show if this is a standalone plot
    if show_plot:
        plt.tight_layout()
        plt.show()
    
    return ax, im  # Return both axes and image object



In [ ]:
animals = ['2792', '3012', '3045', 'F1', 'F2', 'F3', '7132'] #Which are male and which are female?
stimulus = ['Fresh FMU', 'MMU 2', 'feces', 'blood', 'bobcat', 'owl pellet', 'Snake', 'FMU-1', 'FMU-2', 'old FMU'] #Okay so multiple experiments are handles by appending a number, either like -1 or 2?
# animal: 3045
# stimuli: bobcat  Data look really weird. Jen said it was fine, but gonna remove for now.
good_exps = {'3012': ['Fresh FMU', 'MMU 2', 'feces','blood', 'bobcat','Snake' ],
                         '3045':  ['Fresh FMU', 'MMU 2', 'feces', 'blood', 'Snake'],
                         'F1': ['Fresh FMU', 'feces', 'blood', 'bobcat',  'Snake'],
                         'F2': ['Fresh FMU', 'MMU 2','blood','bobcat', 'Snake'],
                         'F3': ['Fresh FMU', 'MMU 2','feces','blood', 'bobcat']}

# Key Info

C_norm: neural activity {animal:{stimulus:[glomerulixframes]}} 10 frames/s (Q: Is there multiple experiments per animal?)

Stimulus_added : frame when stimulus added ()

Stimulus_delivered: frame when stimulus applied to nose of mouse

Stimulus_removed: frame when stimulus removed from arena

Touch_first_mini_frame: first frame of every tapping bout (in miniscope frames)

Touch_first: first time of every tapping bout in 500 Hz recording from touch sensor (I'm gonna ignore this?)

Touch_mini_frame: miniscope frame of every nose tap of that stimulus (This is what we care about)

Touch_per_bout: number of nose taps per bout (How is a bout defined relative to 'Touch_mini_frame'?)

Touch: binary array at 500Hz of touch sensor data. 1= nose tap

In [ ]:

# def our_std(x, mu):
#     return np.sqrt(np.sum((x - mu)**2)/max(1, (len(x)-1)))

# def our_normalization(neural_data, Verbose=1):
#     "Input (neural_data): neural_data.shape = (N x T), N is neurons and T is time steps"
#     "Returns (normalization_values): normalization_values.shape =(N x 2) N is neurons and [mu, std]"
#     neural_data_original = neural_data
#     neural_data = neural_data[:, :2000]
#     pop_sum = np.sum(neural_data, axis=0)
#     pop_med = np.median(pop_sum) 
#     pop_sum_le_med = pop_sum[pop_sum < pop_med] #grab values below the median
#     # Compute the std over the values below the median using the median as the mean.
#     pop_le_med_std = our_std(pop_sum_le_med, pop_med) #pop_le_med_std = np.sqrt(np.sum((pop_sum_le_med - pop_med)**2)/len(pop_sum_le_med)-1)
#     mask = pop_sum < (pop_med + (2 * pop_le_med_std))
#     if Verbose==1:
#         plt.hist(pop_sum, bins=50)
#         plt.axvline(pop_med, c='r', label=f'median = {round(pop_med)}')
#         plt.axvline(pop_med + (2 * pop_le_med_std), c='g', label=f'2$\sigma$ (our method) + median = {round(pop_med + (2 * pop_le_med_std))}')

#         plt.title('Population Sum Histogram')
#         plt.ylabel('Counts')
#         plt.xlabel('Activity Level ')
#         plt.legend()
#         plt.show()


#     # individual level now
#     normalization_values = np.zeros((neural_data.shape[0], 2))
    
#     for i in range(neural_data.shape[0]):
#         individual_neural_data_pop_masked = neural_data[i, mask]
#         individual_mu = np.median(individual_neural_data_pop_masked)
#         individual_non_zero_neural_data = individual_neural_data_pop_masked[individual_neural_data_pop_masked != 0]
#         #temp = individual_non_zero_neural_data[individual_non_zero_neural_data < individual_mu]
#         individual_std = our_std(individual_non_zero_neural_data, individual_mu)
#         normalization_values[i,0] = individual_mu
#         normalization_values[i,1] = individual_std
#         if Verbose == 2:
#             if i % 50 == 0:
                
#                 plt.hist(individual_non_zero_neural_data, bins=50)
#                 plt.axvline(individual_mu, c='r', label=f'median = {(individual_mu)}')

#                 plt.title(f'Individual Histogram Neuron #{i}')
#                 plt.ylabel('Counts')
#                 plt.xlabel('Activity Level ')
#                 plt.legend()
#                 plt.show()
        
#     normalized_data = (neural_data_original - normalization_values[:, [0]]) / (normalization_values[:, [1]] + 1e-4)


#     return normalization_values, normalized_data



# Some Animals have days where a specific neuron is like almost always = 0

In [ ]:

def our_std(x, mu):
    return np.sqrt(np.sum((x - mu)**2)/max(1, (len(x)-1)))


def our_normalization(neural_data, Verbose=1, min_std_threshold='auto', fallback_method='median_mad'):
    """
    Input (neural_data): neural_data.shape = (N x T), N is neurons and T is time steps
    Returns (normalization_values): normalization_values.shape =(N x 2) N is neurons and [mu, std]
    
    Parameters:
    - min_std_threshold: minimum allowed standard deviation to prevent division by tiny numbers
      - float: fixed threshold value
      - 'auto': automatically determine based on data distribution
      - 'percentile_X': use Xth percentile of all computed stds (e.g., 'percentile_5')
      - 'mad_based': use population MAD scaled as threshold
    - fallback_method: what to do when std is too small
      - 'median_mad': use median absolute deviation scaled to match std
      - 'global_std': use a global std estimate across all neurons
      - 'fixed_std': use a fixed minimum std value
    """
    neural_data_original = neural_data
    neural_data = neural_data[:, :2000]
    pop_sum = np.sum(neural_data, axis=0)
    pop_med = np.median(pop_sum) 
    pop_sum_le_med = pop_sum[pop_sum < pop_med]
    pop_le_med_std = our_std(pop_sum_le_med, pop_med)
    mask = pop_sum < (pop_med + (2 * pop_le_med_std))
    
    if Verbose==1:
        plt.hist(pop_sum, bins=50)
        plt.axvline(pop_med, c='r', label=f'median = {round(pop_med)}')
        plt.axvline(pop_med + (2 * pop_le_med_std), c='g', label=f'2$\sigma$ (our method) + median = {round(pop_med + (2 * pop_le_med_std))}')
        plt.title('Population Sum Histogram')
        plt.ylabel('Counts')
        plt.xlabel('Activity Level ')
        plt.legend()
        plt.show()

    # First pass: compute all individual stds to determine dynamic threshold
    all_individual_stds = []
    temp_normalization_data = []
    
    for i in range(neural_data.shape[0]):
        individual_neural_data_pop_masked = neural_data[i, mask]
        individual_mu = np.median(individual_neural_data_pop_masked)
        individual_non_zero_neural_data = individual_neural_data_pop_masked[individual_neural_data_pop_masked != 0]
        individual_std = our_std(individual_non_zero_neural_data, individual_mu)
        
        all_individual_stds.append(individual_std)
        temp_normalization_data.append((individual_mu, individual_std, individual_non_zero_neural_data))
    
    all_individual_stds = np.array(all_individual_stds)
    
    # Determine dynamic threshold
    if isinstance(min_std_threshold, str):
        if min_std_threshold == 'auto':
            # Use a robust method: median - 2*MAD of the std distribution
            std_median = np.median(all_individual_stds)
            std_mad = np.median(np.abs(all_individual_stds - std_median))
            min_std_threshold = max(std_median - 2*std_mad, std_median * 0.1)  # Don't go below 10% of median
            if Verbose >= 1:
                print(f'Auto-determined min_std_threshold: {min_std_threshold:.6f}')
                print(f'  (std_median: {std_median:.6f}, std_mad: {std_mad:.6f})')
        
        elif min_std_threshold.startswith('percentile_'):
            percentile = float(min_std_threshold.split('_')[1])
            min_std_threshold = np.percentile(all_individual_stds, percentile)
            if Verbose >= 1:
                print(f'Using {percentile}th percentile as min_std_threshold: {min_std_threshold:.6f}')
        
        elif min_std_threshold == 'mad_based':
            # Use the population-level MAD as a reference
            all_data_flat = neural_data[mask].flatten()
            all_data_nonzero = all_data_flat[all_data_flat != 0]
            pop_median = np.median(all_data_nonzero)
            pop_mad = np.median(np.abs(all_data_nonzero - pop_median))
            min_std_threshold = pop_mad * 1.4826 * 0.1  # 10% of population std estimate
            if Verbose >= 1:
                print(f'MAD-based min_std_threshold: {min_std_threshold:.6f}')
                print(f'  (pop_mad: {pop_mad:.6f}, scaled: {pop_mad * 1.4826:.6f})')
        
        elif min_std_threshold == 'outlier_detection':
            # Use statistical outlier detection on the std distribution
            Q1 = np.percentile(all_individual_stds, 25)
            Q3 = np.percentile(all_individual_stds, 75)
            IQR = Q3 - Q1
            min_std_threshold = max(Q1 - 1.5 * IQR, np.percentile(all_individual_stds, 1))
            if Verbose >= 1:
                print(f'Outlier-detection min_std_threshold: {min_std_threshold:.6f}')
                print(f'  (Q1: {Q1:.6f}, Q3: {Q3:.6f}, IQR: {IQR:.6f})')
        
        else:
            # Default fallback
            min_std_threshold = 0.01
            if Verbose >= 1:
                print(f'Unknown threshold method, using default: {min_std_threshold}')
    
    # Compute global statistics for fallback
    if fallback_method == 'global_std':
        reasonable_stds = all_individual_stds[all_individual_stds > min_std_threshold]
        global_std_fallback = np.median(reasonable_stds) if len(reasonable_stds) > 0 else min_std_threshold

    # Second pass: apply normalization with determined threshold
    normalization_values = np.zeros((neural_data.shape[0], 2))
    small_std_count = 0
    
    for i in range(neural_data.shape[0]):
        individual_mu, individual_std, individual_non_zero_neural_data = temp_normalization_data[i]
        
        # Handle very small standard deviations
        if individual_std < min_std_threshold:
            small_std_count += 1
            if Verbose >= 1:
                print(f'Neuron {i}: std too small ({individual_std:.6f}), using fallback method: {fallback_method}')
            
            if fallback_method == 'median_mad':
                # Use Median Absolute Deviation (MAD) scaled to match standard deviation
                mad = np.median(np.abs(individual_non_zero_neural_data - individual_mu))
                individual_std = max(mad * 1.4826, min_std_threshold)  # 1.4826 scales MAD to match std for normal distribution
            elif fallback_method == 'global_std':
                individual_std = global_std_fallback
            elif fallback_method == 'fixed_std':
                individual_std = min_std_threshold
            else:
                individual_std = min_std_threshold
        
        normalization_values[i,0] = individual_mu
        normalization_values[i,1] = individual_std
        
        if Verbose == 2:
            if i % 50 == 0:
                plt.hist(individual_non_zero_neural_data, bins=50)
                plt.axvline(individual_mu, c='r', label=f'median = {(individual_mu)}')
                plt.title(f'Individual Histogram Neuron #{i}')
                plt.ylabel('Counts')
                plt.xlabel('Activity Level ')
                plt.legend()
                plt.show()
    
    if Verbose >= 1 and small_std_count > 0:
        print(f'Warning: {small_std_count}/{neural_data.shape[0]} neurons had std < {min_std_threshold:.6f}')
        print(f'Used fallback method: {fallback_method}')
        
    # Show distribution of standard deviations for diagnostics
    if Verbose >= 1:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.hist(all_individual_stds, bins=50, alpha=0.7, label='Original STDs')
        plt.axvline(min_std_threshold, color='red', linestyle='--', label=f'Threshold: {min_std_threshold:.6f}')
        plt.xlabel('Standard Deviation')
        plt.ylabel('Count')
        plt.title('Distribution of Individual STDs')
        plt.legend()
        plt.yscale('log')
        
        plt.subplot(1, 2, 2)
        final_stds = normalization_values[:, 1]
        plt.hist(final_stds, bins=50, alpha=0.7, label='Final STDs', color='green')
        plt.axvline(min_std_threshold, color='red', linestyle='--', label=f'Threshold: {min_std_threshold:.6f}')
        plt.xlabel('Standard Deviation')
        plt.ylabel('Count')
        plt.title('Distribution of Final STDs')
        plt.legend()
        plt.yscale('log')
        plt.tight_layout()
        plt.show()
    
    # Apply normalization with the robust std values
    normalized_data = (neural_data_original - normalization_values[:, [0]]) / normalization_values[:, [1]]
    
    return normalization_values, normalized_data
all_neural_data = load_pickle('C_norm.pkl')
cleaned_data = clean_neural_data(all_neural_data, good_exps, zero_threshold=0.5)
use_first = False
normalize = True
counter = 0
if use_first: 
    all_touch = load_pickle('touch_first_mini_frame.pkl')
else: 
    all_touch = load_pickle('touch_mini_frame.pkl')
for animal in cleaned_data:
    for stimuli in cleaned_data[animal]:
        
        print(f'animal: {animal}')
        print(f'stimuli: {stimuli}')
        try: 
            neural_data = cleaned_data[animal][stimuli]
        except KeyError as ke:
            print(f'this animal {animal}, does not have neural data with this stimuli: {stimuli}')
            continue

        try: 
            touches = all_touch[animal][stimuli]
        except KeyError as ke:
            print(f'this animal {animal}, does have neural data but does not have touch sensor data for this stimuli: {stimuli}')
            continue
        if normalize:
            verbose_level = 1 if counter == 0 else 0
            _, neural_data_normalized = our_normalization(neural_data, Verbose=verbose_level, 
                                             min_std_threshold='auto', 
                                             fallback_method='median_mad')
            counter +=1
            fig, axes = plt.subplots(1, 2, figsize=(12, 8))
            vis_data = neural_data_normalized
            vis_data[vis_data < 0] = 0
            vis_data = np.sqrt(vis_data)
            # Plot both heatmaps and get the image objects
            ax0, im0 = plot_heat_touch(vis_data, touches, 
                                     title='Normalized Neural Data', ax=axes[0])
            ax1, im1 = plot_heat_touch(neural_data, touches, 
                                     title='Raw Neural Data', ax=axes[1])
            
            # Add colorbars for both subplots
            plt.colorbar(im0, ax=axes[0], label='Normalized Activity')
            plt.colorbar(im1, ax=axes[1], label='Raw Activity')
            
            plt.tight_layout()
            plt.show()
        # offset = np.nanmin(neural_data)
        # neural_data = np.sqrt(neural_data - offset)
        # neural_data = neural_data + np.sqrt(np.abs(offset))
        
        


# Clustering

In [ ]:
for mouse, stimuli in cleaned_data.items():
    print(mouse)

In [ ]:
# Concatenate along time dimensions (data will be T x N)
normalized_data = {}
for mouse_name, mouse in cleaned_data.items():
    normalized_data[mouse_name] = {}
    for stim_name, responses in mouse.items():
        _, normalized_data[mouse_name][stim_name] = our_normalization(responses, Verbose=verbose_level, 
                                             min_std_threshold='auto', 
                                             fallback_method='median_mad')
data = [x for x in normalized_data['3012'].values()]
data = np.hstack(data)
data_vis = data
data_vis[data_vis < 0] = 0
data_vis = np.sqrt(data_vis)
plt.imshow(data_vis, extent=[0, data.shape[1]/HZ/25, 0, data.shape[0]])
plt.colorbar()
plt.show()

data2 = [x for x in cleaned_data['3012'].values()]
data2 = np.hstack(data2)
plt.imshow(data2, extent=[0, data2.shape[1]/HZ/25, 0, data2.shape[0]])
plt.colorbar()
plt.show()

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_

In [ ]:
from sklearn.decomposition import PCA
data_centered = data - np.mean(data, axis=1, keepdims=True)
# plt.imshow(data_centered, extent=[0, data_centered.shape[1]/HZ/25, 0, data_centered.shape[0]])
# plt.colorbar()

data_cov = (data_centered @ data_centered.T) / data_centered.shape[1]
# plt.imshow(data_cov)
# plt.colorbar()
vals, vectors = np.linalg.eig(data_cov)
# plt.bar(np.arange(0,vals.shape[0]), vals)
print(data_centered.shape)
print(vectors.shape)
for i in range(1):
    plt.plot(data_centered.T @ vectors[:, i])

for i in range(10):
    vectors_sorted = np.argsort(vectors[:, i])
    plt.imshow(data_vis[vectors_sorted, :], extent=[0, data_centered.shape[1]/HZ/25, 0, data_centered.shape[0]])
    plt.show()

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(data.T)
H = model.components_
print(H.shape)


In [ ]:
plt.plot(W)
plt.show()
for i in range(10):
    vectors_sorted = np.argsort(H[i, :])
    plt.imshow(data_vis[vectors_sorted[H[i, vectors_sorted] > np.mean(H[i,:])], :], aspect='auto')
    plt.show()

In [ ]:
all_neural_data = load_pickle('C_norm.pkl')
all_exps = {}

for animal in good_exps:
    for stimuli in good_exps[animal]:
        # Get the data for this experiment
        data = all_neural_data[animal][stimuli]
        
        # Calculate zero proportion for each neuron
        zero_proportions = []
        for neuron_number in range(data.shape[0]):
            # Count zeros and calculate proportion
            zero_count = np.sum(data[neuron_number, :] == 0)
            zero_proportion = zero_count / data.shape[1]
            zero_proportions.append(zero_proportion)
        
        # Store the results using tuple as key
        all_exps[(animal, stimuli)] = zero_proportions
all_exps
for animal, stimuli in list(all_exps.keys()):
    plt.hist(all_exps[(animal, stimuli)], bins=50)
    plt.xlabel('Zero Proportion')
    plt.ylabel('Number of Neurons')
    plt.title(f'Distribution of Zero Proportions - {animal}, {stimuli}')
    plt.show()

In [ ]:
print(list(all_exps.keys()))
for animal, stimuli in list(all_exps.keys()):
    print(animal, stimuli )

# Working with individual stuff


In [ ]:
def plot_hist_normalizer(neural_data):
    pop_sum = np.sum(neural_data, axis=0)
    pop_med = np.median(pop_sum)

    pop_le_med_std_theres = pop_sum[pop_sum < pop_med].std()

    pop_sum_le_med = pop_sum[pop_sum < pop_med] 
    pop_le_med_std_ours = np.sqrt(np.sum((pop_sum_le_med - pop_med)**2)/(len(pop_sum_le_med)-1))

    pop_le_med_2med_ours = pop_med + (2 * pop_le_med_std_ours)
    pop_le_med_2med_theres = pop_med + (2 * pop_le_med_std_theres)


    plt.hist(pop_sum, bins=50)
    plt.axvline(pop_med, c='r', label=f'median = {round(pop_med)}')
    plt.axvline(pop_le_med_2med_ours, c='g', label=f'2$\sigma$ (our method) + median = {round(pop_le_med_2med_ours)}')
    plt.axvline(pop_le_med_2med_theres, c='k', label=f'2$\sigma$ + median = {round(pop_le_med_2med_theres)}')

    plt.title('Population Sum Histogram')
    plt.ylabel('Counts')
    plt.xlabel('Activity Level ')
    plt.legend()
    plt.show()



So sometimes we have the neural data but not the tapping data? Yes!

If a glomeruli is present in animal A with stimulus presentation B, is that same glomeruli present (and indexed the same) throughout each experiment? 